# LAB 5: Text classification with Naive Bayes

Objectives:

* Train and evaluate Naive Bayes text classifiers
* Cross-validation
* Hyperparameter search

In [ ]:
import numpy as np
import pandas as pd
from cytoolz import *
from tqdm.auto import tqdm

tqdm.pandas()

### Load and preprocess data

In [ ]:
df = pd.read_parquet(
    "s3://ling583/rcv1-politics.parquet", storage_options={"anon": True}
)

In [ ]:
import spacy

nlp = spacy.load(
    "en_core_web_sm",
    exclude=["tagger", "parser", "ner", "lemmatizer", "attribute_ruler"],
)

In [ ]:
def tokenize(text):
    doc = nlp.tokenizer(text)
    return [t.norm_ for t in doc if t.is_alpha]

In [ ]:
df["tokens"] = df["text"].progress_apply(tokenize)

---

### Baseline dummy classifier

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import StratifiedKFold, cross_val_predict, cross_val_score

Set up five-fold cross-validation. We'll use the same training/test splits for all our experiments so the results will be easier to compare.

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=5432)

In [ ]:
baseline = DummyClassifier()
cross_val_score(baseline, df["tokens"], df["pol"], cv=cv)

In [ ]:
predicted = cross_val_predict(baseline, df["tokens"], df["pol"], cv=cv)
print(classification_report(df["pol"], predicted))

----

### Bernoulli Naive Bayes

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.pipeline import make_pipeline

Set up a pipeline: first convert tokenized text into feature vectors, then apply naive Bayes classifier

In [ ]:
bnb = make_pipeline(CountVectorizer(analyzer=identity), BernoulliNB())

In [ ]:
predicted = cross_val_predict(bnb, df["tokens"], df["pol"], cv=cv, n_jobs=-1)
print(classification_report(df["pol"], predicted))

---

### Hyperparameter search

In [ ]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:34795")
client

In [ ]:
from dask_ml.model_selection import RandomizedSearchCV
from scipy.stats.distributions import loguniform, randint, uniform

In [ ]:
from warnings import simplefilter
simplefilter(action="ignore", category=FutureWarning)

In [ ]:
bnb = make_pipeline(CountVectorizer(analyzer=identity), BernoulliNB())

In [ ]:
%%time

search = RandomizedSearchCV(
    bnb, {"bernoullinb__alpha": loguniform(1e-10, 10.0)}, n_iter=25, scoring="f1"
)
search.fit(df["tokens"], df["pol"])

In [ ]:
search.best_params_

In [ ]:
search.best_score_

In [ ]:
cv_results = pd.DataFrame(search.cv_results_)

In [ ]:
cv_results.plot(
    "param_bernoullinb__alpha", "mean_test_score", kind="scatter", logx=True
)

In [ ]:
%%time

search = RandomizedSearchCV(
    bnb,
    {
        "countvectorizer__min_df": randint(1, 5),
        "bernoullinb__alpha": loguniform(1e-10, 1e-5),
    },
    n_iter=25,
    scoring="f1",
)
search.fit(df["tokens"], df["pol"])

In [ ]:
search.best_params_, search.best_score_

In [ ]:
cv_results = pd.DataFrame(search.cv_results_)

In [ ]:
cv_results.plot("param_countvectorizer__min_df", "mean_test_score", kind="scatter")

In [ ]:
cv_results.plot(
    "param_bernoullinb__alpha", "mean_test_score", kind="scatter", logx=True
)

In [ ]:
cv_results.plot(
    "param_bernoullinb__alpha",
    "mean_test_score",
    kind="scatter",
    logx=True,
    c="param_countvectorizer__min_df",
    colormap="Set1",
)

In [ ]:
%%time

search = RandomizedSearchCV(
    bnb,
    {
        "countvectorizer__min_df": [1],
        "countvectorizer__max_df": uniform(0.5, 0.5),
        "bernoullinb__alpha": loguniform(1e-10, 1e-5),
    },
    n_iter=25,
    scoring="f1",
)
search.fit(df["tokens"], df["pol"])

In [ ]:
search.best_params_, search.best_score_

In [ ]:
cv_results = pd.DataFrame(search.cv_results_)
cv_results.plot("param_countvectorizer__max_df", "mean_test_score", kind="scatter")

---

### Evaluate final model

In [ ]:
bnb.set_params(
    bernoullinb__alpha=1e-10, countvectorizer__min_df=1, countvectorizer__max_df=0.73
)

In [ ]:
predicted = cross_val_predict(bnb, df["tokens"], df["pol"], cv=cv, n_jobs=-1)
print(classification_report(df["pol"], predicted))